## Item2Vec

通过基于相似item做推荐的一种方法，将用户观看（喜欢）的电影id集合看作句子，将全部用户的item集合作为整个文本，使用word2vec训练出item对应的词向量，然后计算相似度，获得相似item。

In [2]:
# coding=utf-8
import os
import pandas as pd
import numpy as np
import operator
import time
import gensim
from sklearn.metrics.pairwise import cosine_similarity

### 读取数据

In [4]:
movies = pd.read_csv('/home/zwj/Desktop/recommend/movielens/moive_database/v1/v1_movie_info.csv',\
                        usecols=[0, 1, 6], header=0, names=['movieId', 'title', 'genres'])

df_train = pd.read_csv('/home/zwj/Desktop/recommend/movielens/moive_database/v1/v1_train.csv', \
                       usecols=[0, 1, 2])  # userId, movieId, rating

### 设置参数

In [58]:
# 推荐电影数
reco_num = 10
# 加权求和计算的相似项个数
sim_num = 10
# word2vec训练的模型文件
item2vec_model_file = './item2vec.model'
# 词向量维度
vec_dim = 128
# word2vec模型训练迭代数
iterations = 1000

### 功能函数

获取全局热门电影

In [16]:
def getHotItem(df_train, N=5):
    """
    param:
        df_train: 训练数据集 type:dataframe
        N: 推荐的电影数
    return: 
        hot_rank: 该用户的推荐热门电影列表 type:dict, key:user, value:dict, key:item, value:sim
    """

    item_count = df_train.groupby('movieId')['rating'].count().sort_values(ascending=False)

    hot_rank = {}

    r = 0
    for item_id in item_count[0:N].index:
        hot_rank[item_id] = 1 - 0.01 * r
        r += 1
    return hot_rank

hot_rank = getHotItem(df_train, reco_num)

hot_rank

{1: 0.94,
 50: 1.0,
 56: 0.91,
 100: 0.97,
 174: 0.92,
 181: 0.99,
 258: 0.98,
 288: 0.95,
 294: 0.96,
 300: 0.9299999999999999}

生成用户-电影排列表

In [17]:
def userItemDict(data):
    """
    param:
        data: lsit [user, item, rating]
    return:
        user_item: 用户-电影排列表 type:dict, key:user, value:dict, key:item, value:rate
    """
    user_item = {}
    for user, item, rate in data:
        if user not in user_item:
            user_item[user] = {}
        user_item[user].update({item: rate})
    return user_item

# 生成user-tiem排列表
user_item = userItemDict(df_train.values)

user_item[105]

{264.0: 2.0,
 270.0: 5.0,
 271.0: 2.0,
 288.0: 4.0,
 302.0: 5.0,
 313.0: 5.0,
 324.0: 4.0,
 327.0: 4.0,
 340.0: 3.0,
 343.0: 2.0,
 347.0: 3.0,
 690.0: 3.0,
 751.0: 2.0,
 880.0: 3.0}

获得训练语料（itemword），用户在历史数据中观看的（喜欢的）电影集合作为itemWord，每个用户对应的list可看作是一个句子。

In [18]:
def itemWord(data):
    """
    param:
        data: type:ndarray [[user, item, rating],[...]]
    """
    # 将data转成如下字典格式，注意itemid存储的是字符串
    # {userid1: ['itemid1', 'itemid2'], userid2: [...], ...}
    user_itemWord = {}
    for user, item, rate in data:
        if user not in user_itemWord:
            # int()是为了最后去掉小数点.0的部分
            user_itemWord[int(user)] = []
        # 判断，电影评分大于3的，归到感兴趣的影片集中
        if rate > 3.0:
            # str()将itemid转成字符串，因为join的对象必须是字符串
            user_itemWord[user].append(str(int(item)))
    # 保存user_itemWord中values部分['itemid1', 'itemid2']，每个用户的item集合为一行，看作一个句子
    itemwords = user_itemWord.values()
    return itemwords
    
train_corpus = itemWord(df_train.values)

train_corpus[3]

['258', '271', '300', '301', '324', '327', '329', '360', '362']

训练word2vec模型

In [44]:
def trainWord2vecModel(train_corpus, save_model):
    # 训练模型
    model = gensim.models.Word2Vec(train_corpus, size=vec_dim, window=2, sample=1e-4, \
                                   negative=5, hs=0, sg=1, iter=iterations, min_count=1)
    # 保存模型
    model.save(save_model)
    return model

model_item2vec = trainWord2vecModel(train_corpus, item2vec_model_file)

加载model

In [ ]:
# model_item2vec = gensim.models.Word2Vec.load(item2vec_model_file)

获取item对应的向量集合

In [53]:
def getItemVec(model, item_set):
    item_vec = {}
    for itemid in item_set:
        try:
            item_vec[itemid] = model.wv[str(itemid)]
        except:
            # print('itemid:'+ str(itemid) + ' not found')
            continue
    return item_vec

item_vec = getItemVec(model_item2vec, df_train['movieId'].unique())

计算item_vec的相似度

In [54]:
def calItemSim(item_vec, item_id, K=10):
    """
    param:
        item_vec: type:dict key:userid values:词向量
        item_id: 电影id
        K: 考虑相似item的个数，默认为10
    return: 
        score：返回与item_id相似的id及相似度，type:list [(itemid1, sim1), (itemid2, sim2)]
    """
    # 如果item_id不在item向量字典中，返回空
    if item_id not in item_vec:
        return

    score = {}
    fix_item_vec = item_vec[item_id]

    # 遍历向量字典中的所有itemid，与该item_id计算相似度
    for tmp_itemid in item_vec:
        # 如果是自己，则跳过
        if tmp_itemid == item_id:
            continue
        tmp_itemvec = item_vec[tmp_itemid]

        # 计算cosine相似度方法1，使用函数
        tmp_sim = np.concatenate((fix_item_vec.reshape(1, -1), tmp_itemvec.reshape(1, -1)), axis=0)
        # cosine_similarity返回列表中[0, 1]位置就是需要的相似度的值
        score[tmp_itemid] = cosine_similarity(tmp_sim)[0, 1]

        # 计算cosine相似度方法2，直接计算
        # fenmu = np.linalg.norm(fix_item_vec) * np.linalg.norm(tmp_itemvec)
        # if fenmu == 0:
        #     score[tmp_itemid] = 0
        # else:
        #     score[tmp_itemid] = round(np.dot(fix_item_vec, tmp_itemvec)/fenmu, 3)

    # 对item向量相似度字典进行排序，返回前K个向此向量
    score = sorted(score.items(), key=operator.itemgetter(1), reverse=True)[0:K]
    return score

推荐系统

In [55]:
def recommendation(user_item, user_id, hot_rank, item_vec, K, R):
    """
    param:
        user_item: 用户-电影排列表 type:dict, key:user, value:dict, key:item, value:rate
        user_id: 推荐的用户id
        hot_rank: 热门电影列表, type:dict, key:user, value:dict, key:item, value:sim
        item_vec: item词向量
        K: 前K个最相似用户
        R: 推荐列表中电影个数
    return: 
        rank_sorted：该用户的推荐电影列表 type:dict, key:user, value:dict, key:item, value:sim
    """
    # 存储用户推荐电影
    rank = {}
    # 开辟用户空子字典 ('rank: ', {user_id: {}})
    rank.setdefault(user_id, {})

    # 如果该用户不在训练集中，则推荐热门电影
    if user_id not in user_item:
        print('user {} not in trainset, give hot rank list'.format(user_id))
        rank[user_id] = hot_rank
    else:
        # 用户已观看的电影集合
        watched_item_list = user_item[user_id]
        # 该指标用于判断，如果userid用户中观看的电影itemid均不再item词向量中，则没有相似度判断依据，返回热门电影
        item_in_vec = False
        # item_i:项目号， ri:对应的评分（兴趣度）
        for item_i, ri in watched_item_list.items():
            if item_i not in item_vec:
                continue
            # 只要存在一个及以上的item在item词向量中，则表示为True，不用返回热门电影
            item_in_vec = True
            # 获得itemid的相似item集
            simMovie = calItemSim(item_vec, item_i, K)
            # 遍历该item集合，推荐的电影在这个集合中选择
            for item_j, simj in simMovie:
                # 如果item是已观看过的，则跳过
                if item_j in watched_item_list:
                    continue

                rank[user_id].setdefault(item_j, 0)
                # 电影推荐度 = 用户评分（或者兴趣度）* 电影相似度
                rank[user_id][item_j] += ri * simj
        # userid用户中观看的电影itemid均不再item词向量中，则没有相似度判断依据，返回热门电影
        if item_in_vec == False:
            rank[user_id] = hot_rank

    # 对推荐的电影排序，返回前R部电影
    rank_sorted = {}
    rank_sorted[user_id] = sorted(rank[user_id].items(), key=operator.itemgetter(1), reverse=True)[0:R]

    return rank_sorted

### 电影推荐

In [59]:
# 输入用户ID
user_id = input('Please input your user ID:')

recom_list = recommendation(user_item, user_id, hot_rank, item_vec, sim_num, reco_num)
# 输出推荐电影
print('\nRecommended movies：\n')

ranknum = 0
for idx, rate  in recom_list.values()[0]:
    ranknum += 1
    print("%d. %s" %(ranknum, movies[movies['movieId'] == idx]['title'].values[0]))
    print '-'*50


Please input your user ID:105

Recommended movies：

1. Apt Pupil
--------------------------------------------------
2. Air Force One
--------------------------------------------------
3. Steel
--------------------------------------------------
4. Face/Off
--------------------------------------------------
5. Critical Care
--------------------------------------------------
6. Contact
--------------------------------------------------
7. Ulee's Gold
--------------------------------------------------
8. Kiss the Girls
--------------------------------------------------
9. Good Will Hunting
--------------------------------------------------
10. Jackie Brown
--------------------------------------------------
